In [8]:
import instaloader
import pandas as pd
import nltk
import pytesseract

In [2]:
L = instaloader.Instaloader()

USERNAME = "faizgram"
SESSION_FILE = "session-faizgram"

try:
    L.load_session_from_file(username=USERNAME, filename=SESSION_FILE)
except FileNotFoundError as e:
    print("Session file not found. Please log in manually using Instaloader CLI.")
    exit()
except instaloader.ConnectionException as e:
   print(f"Connection exception {e}")
   exit()

Loaded session from session-faizgram.


In [3]:
def by_hashtag(hashtag, limit, profile_name=None, location=None):
    post_count = 0
    hashtag_posts = pd.DataFrame(columns=["Thumb", "Caption", "Comment"])
    thumbs = []
    captions = []
    comments = []
    dates = []

    posts = L.get_hashtag_posts(hashtag)

    for post in posts:
        if post.caption:
            thumbs.append(post.url)
            captions.append(post.caption)
            comments.append(post.comments)
            dates.append(post.date)
            # print(f"Extracted caption: {post.caption}")
        
        post_count += 1
        if post_count >= limit:
            break

    hashtag_posts["Thumb"] = thumbs
    hashtag_posts["Caption"] = captions
    hashtag_posts["Date"] = dates
    hashtag_posts["Comment"] = comments

    return hashtag_posts

In [4]:
posts = by_hashtag("unikl", 5, None, None)

In [5]:
posts

,Thumb,Caption,Comment,Date
0,https://instagram.fkul4-3.fna.fbcdn.net/v/t51....,❤️❤️RUN❤️❤️\nSouthern Bundle Collection\n\nNB ...,0,2024-12-10 13:39:19
1,https://instagram.fkul4-5.fna.fbcdn.net/v/t51....,❤️❤️COURT❤️❤️\nSouthern Bundle Collection\n\nA...,0,2024-12-10 12:35:53
2,https://instagram.fkul4-4.fna.fbcdn.net/v/t39....,❤️❤️COURT❤️❤️\nSouthern Bundle Collection\n\nM...,0,2024-12-10 11:50:29
3,https://instagram.fkul4-5.fna.fbcdn.net/v/t39....,Betul ke apa yang kita fikir akan jadi doa? \n...,0,2024-12-10 11:39:19
4,https://instagram.fkul4-4.fna.fbcdn.net/v/t51....,Posted @withregram • @cmt.uniklmiit 🌟 Congratu...,1,2024-12-10 11:32:21


### Preprocessing

In [ ]:
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from transformers import pipeline

from PIL import Image
import requests
from io import BytesIO

# Tesseract path (Windows 10/11)
tesseract_path = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Unix/Linux

# MacOS

pytesseract.pytesseract.tesseract_cmd = tesseract_path

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mohdf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mohdf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mohdf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
bs4 is not installed, `malaya.text.function.remove_html_tags` will use regex


ModuleNotFoundError: No module named 'torch'

In [17]:
img_texts = []
for img_url in posts["Thumb"]:
    response = requests.get(img_url)
    img_text = pytesseract.image_to_string(Image.open(BytesIO(response.content)))
    print(f"Extracted text: {img_text}")
    img_texts.append(img_text)

posts["Thumb Text"] = img_texts

Extracted text: 
Extracted text: 
Extracted text: 
Extracted text: 
Extracted text: xX XX KX X XK XK X




In [18]:
posts

,Thumb,Caption,Comment,Date,Thumb Text
0,https://instagram.fkul4-3.fna.fbcdn.net/v/t51....,❤️❤️RUN❤️❤️\nSouthern Bundle Collection\n\nNB ...,0,2024-12-10 13:39:19,
1,https://instagram.fkul4-5.fna.fbcdn.net/v/t51....,❤️❤️COURT❤️❤️\nSouthern Bundle Collection\n\nA...,0,2024-12-10 12:35:53,
2,https://instagram.fkul4-4.fna.fbcdn.net/v/t39....,❤️❤️COURT❤️❤️\nSouthern Bundle Collection\n\nM...,0,2024-12-10 11:50:29,
3,https://instagram.fkul4-5.fna.fbcdn.net/v/t39....,Betul ke apa yang kita fikir akan jadi doa? \n...,0,2024-12-10 11:39:19,
4,https://instagram.fkul4-4.fna.fbcdn.net/v/t51....,Posted @withregram • @cmt.uniklmiit 🌟 Congratu...,1,2024-12-10 11:32:21,xX XX KX X XK XK X\n\n


In [34]:
import malaya

ModuleNotFoundError: No module named 'torch'

In [ ]:
def preprocess_text(text):
    # this uses NLTK
    # tokens = word_tokenize(text.lower())

    # this uses malaya toolkit
    tokens = malaya.tokenizer.Tokenizer(text.lower())

    # remove alphanumeric word(s)
    filtered_tokens = [word for word in tokens if word.isalnum()]
    return ' '.join(filtered_tokens)

In [11]:
def combine_text_data(image_texts, description_texts):
    all_texts = image_texts + description_texts
    preprocessed_texts = [preprocess_text(text) for text in all_texts]
    return preprocessed_texts

In [33]:
texts = []
for text in posts["Caption"]:
   texts.append(preprocess_text(text))

NameError: name 'malaya' is not defined

In [30]:
texts

['southern bundle collection nb 411 all so size kondisi rm 128 free pos semenanjung sbc9uk souternbundle2 asicsbundle mizunobundle b runningshoes unikl uitm upm utm hokabundle tldm tudm pdrm ukm politeknik ipts ansara kasutbundle jlj polis ramd wataniah upsi ipg mitec uia ukm ipta',
 'southern bundle collection as1cs bladelyte size 27cm kondisi rm 110 free pos semenanjung sbc8uk souternbundle2 asicsbundle mizunobundle b runningshoes unikl uitm upm utm hokabundle tldm tudm pdrm ukm politeknik ipts ansara kasutbundle jlj polis ramd wataniah upsi ipg mitec uia ukm ipta',
 'southern bundle collection m1zuno wave lightning z5 size kondisi rm 178 free pos semenanjung sbc7uk souternbundle2 asicsbundle mizunobundle b runningshoes unikl uitm upm utm hokabundle tldm tudm pdrm ukm politeknik ipts ansara kasutbundle jlj polis ramd wataniah upsi ipg mitec uia ukm ipta',
 'betul ke apa yang kita fikir akan jadi doa otak kita ni allah cipta untuk kita gunakan sebaiknya bila kita fikir sesuatu soalan 